# Visualizzazione avanzata

## Cosa impareremo

* Visualizzazione dei dati in scala logaritmica
* Calcolo degli attivi
* Visualizzazioni alternative (diagrammi a barre)
* Estrazione dei dati incrementali
    * Estrazione semplice
    * Filtraggio dei dati
* Rappresentazioni alternative (incremento in funzione del numero dei casi)

## Caricamento dei dati e definizione di funzioni (dal modulo precedente)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import scipy.special

# Dati mondo

mondo_URL_base = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'
file_confermati = 'time_series_covid19_confirmed_global.csv'
file_morti = 'time_series_covid19_deaths_global.csv'
file_guariti = 'time_series_covid19_recovered_global.csv'

print('Lettura dati mondiali...')
confermati_mondo = pd.read_csv(mondo_URL_base + file_confermati, index_col = 'Country/Region').sort_index().drop(columns = ['Province/State', 'Lat', 'Long'])
morti_mondo = pd.read_csv(mondo_URL_base + file_morti, index_col = 'Country/Region').sort_index().drop(columns = ['Province/State', 'Lat', 'Long'])
guariti_mondo = pd.read_csv(mondo_URL_base + file_guariti, index_col = 'Country/Region').sort_index().drop(columns = ['Province/State', 'Lat', 'Long'])
print('Fatto')

def dati_paese_singolo(dati, paese):
    dati_uscita = dati.loc[paese]
    if len(dati_uscita.shape) == 2:
        dati_uscita = dati_uscita.sum()
    return dati_uscita

def dati_paese(paese):
    return dati_paese_singolo(confermati_mondo, paese), dati_paese_singolo(guariti_mondo, paese), dati_paese_singolo(morti_mondo, paese)

# Dati italiani

italia_URL_base = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/'
formato_file_gioraliero = 'dpc-covid19-ita-regioni-%Y%m%d.csv'

data_inizio = datetime.date(2020,2,24) # inizio il 24 febbraio 2020

lista_dati = []

print('Letture dati italiani...')
data_corrente = data_inizio
while data_corrente < datetime.date.today():
    file = data_corrente.strftime(formato_file_gioraliero)
    print(f'Lettura di {file}')
    lista_dati.append(pd.read_csv(italia_URL_base + file, index_col=['denominazione_regione', 'data']).sort_index())
    data_corrente += datetime.timedelta(days=1) # aggiunge un giorno alla data considerata
    
dati_italia_per_regione = pd.concat(lista_dati) # Aggrega i dati
print('Fatto')

def dati_regione(regione):
    confermati = dati_italia_per_regione.loc[regione]['totale_casi'] # seleziona le righe "Regione" e la colonna "totale_casi"
    guariti = dati_italia_per_regione.loc[regione]['dimessi_guariti']
    morti = dati_italia_per_regione.loc[regione]['deceduti']
    return confermati, guariti, morti

## La scala logaritmica

L'epidemia non controllata segue un andamento di espansione di tipo esponenziale, cioè se il primo paziente (paziente zero) infetta due persone, a loro volta queste due ne infetteranno due a testa, e quindi quattro, dopodiché otto, sedici, eccetera... Questo andamento esponenziale è rappresentato da questa curva caratteristica:

In [ ]:
plt.plot(np.exp(np.linspace(0,10)))

Man mano che le misure di contenimento iniziano a funzionare, o, in mancanza di queste, il virus inizia a colpire una porzione significativa della popolazione (ma stiamo parlando già di milioni di individui!), l'andamento esponenziale inizia a rallentare, e l'epidemia tenderà ad assumere la forma di una funzione a "S" detta logistica. La prima parte di questa curva può essere approssimata con un'esponenziale, la seconda parte con una funzione lineare, ed infine vi è una parte di "saturazione" di tipo ancora esponenziale decrescente.

In [ ]:
plt.plot(scipy.special.expit(np.linspace(-10,10)))

Si dice che le misure di contenimento iniziano a funzionare quando l'accrescimento della curva inizia a deviare dall'esponenziale. A occhio, questo è difficile a dirsi, però l'esponenziale ha una caratteristica: in una scala logaritmica, essa è rappresentata da una retta. La scala logaritmica è una scala in cui gli indici non crescono in maniera lineare, ma in cui la distanza tra 1 e 10 è uguale alla distanza tra 10 e 100, 100 e 1000 e così via. In pratica, i numeri grandi sono compressi e i numeri piccoli sono dilatati. In Python, la scala logaritmica si imposta molto semplicemente con ```yscale```

In [ ]:
plt.plot(np.exp(np.linspace(0,10)))
plt.yscale('log')
plt.title('Esponenziale in scala logaritmica')
plt.figure()
plt.plot(scipy.special.expit(np.linspace(-10,10)))
plt.yscale('log')
plt.title('Logistica in scala logaritmica')

Vediamo adesso come la scala logaritmica aiuta nella visualizzazione dei dati reali.

In [ ]:
confermati_italia, guariti_italia, morti_italia = dati_paese('Italy')
confermati_uk, guariti_uk, morti_uk = dati_paese('United Kingdom')
confermati_cina, guariti_cina, morti_cina = dati_paese('China')

plt.plot(confermati_italia, label='Italia')
plt.plot(confermati_uk, label='UK')
plt.title('Casi confermati, scala lineare')
plt.legend()
plt.figure()
plt.plot(confermati_italia, label='Italia')
plt.plot(confermati_uk, label='UK')
plt.title('Casi confermati, scala logaritmica')
plt.yscale('log')
plt.legend()

In questo grafico si vede come l'Italia sia rimasta nella fase puramente esponenziale relativamente poco, mentre il Regno Unito ha ritardato molto nell'implementazione delle misure di contenimento, nonostante abbia avuto una crescita iniziale più lenta e i numeri assoluti siano molto diversi.

Vediamo adesso come alcune regioni italiane appaiono in scala lineare e logaritmica.

In [ ]:
plt.plot(dati_regione('Lombardia')[0], label='Lombardia')
plt.plot(dati_regione('Toscana')[0], label='Toscana')
plt.plot(dati_regione('Puglia')[0], label='Puglia')
plt.title('Scala lineare')
plt.legend()

plt.figure()
plt.plot(dati_regione('Lombardia')[0], label='Lombardia')
plt.plot(dati_regione('Toscana')[0], label='Toscana')
plt.plot(dati_regione('Puglia')[0], label='Puglia')
plt.title('Scala logaritmica')
plt.yscale('log')
plt.legend()

## Calcolo dei pazienti attivi

Considerare i casi confermati è utile perché sappiamo abbastanza bene come modellare questo tipo di crescita; però ha il difetto di essere una curva che non decresce mai, e quindi non ha mai un "picco".

Può essere quindi utile osservare anche i pazienti attivi, cioè il numero dei pazienti infetti in un qualsiasi momento. A un certo punto, le guarigioni (e, sfortunatemente, i decessi) supereranno in numero i nuovi contagi, e quindi questo numero di pazienti attivi inizierà a diminuire.

I pazienti attivi sono semplicemente i pazienti confermati meno i guariti ed i deceduti.

In [ ]:
attivi_italia = confermati_italia - guariti_italia - morti_italia
attivi_cina = confermati_cina - guariti_cina - morti_cina

plt.plot(attivi_cina, label='Cina')
plt.plot(attivi_italia, label='Italia')
plt.legend()
plt.title('Attivi, scala lineare')

plt.figure()
plt.plot(attivi_cina, label='Cina')
plt.plot(attivi_italia, label='Italia')
plt.legend()
plt.yscale('log')
plt.title('Attivi, scala logaritmica')

Personalmente, preferisco la rappresentazione lineare per questo tipo di variabile, perché la rappresentazione logaritmica tende ad allargare il picco (visto che "schiaccia" i valori alti), ma è principalmente questione di gusti.

## Rappresentazioni integrate

A volte può essere utile rappresentare più valori sullo stesso grafico, ed un caso classico è quello di rappresentare i valori di attivi, guariti e deceduti cumulati tra di loro. Per questo possiamo usare la funzione ```bar```. Siccome questa visualizzazione è pensata per istogrammi, dobbiamo anche esplicitare l'asse delle ascisse (x).

In [ ]:
x = np.arange(len(attivi_italia))
plt.bar(x, attivi_italia, label='attivi')
plt.bar(x, morti_italia, bottom=attivi_italia, label='deceduti')
plt.bar(x, guariti_italia, bottom=(attivi_italia+morti_italia), label='guariti')
plt.legend()
plt.xlim(35, None) # Zoom sul grafico

Possiamo incapsulare i comandi precedenti in una funzione:

In [ ]:
def grafico_integrato(confermati, guariti, morti):
    attivi = confermati-guariti-morti
    x = np.arange(len(attivi))
    plt.bar(x, attivi, width=1.0, label='attivi')
    plt.bar(x, morti, width=1.0, bottom=attivi, label='deceduti')
    plt.bar(x, guariti, width=1.0, bottom=(attivi+morti), label='guariti')
    plt.legend()


Possiamo adesso chiamare la funzione ```grafico_integrato``` direttamente con i dati di uscita di ```dati_paese``` o ```dati_regione```. Notare l'uso dell'asterisco di fronte alla chiamata di funzione. Questa è una funzione Python che permette di usare i tre dati di ritorno di dati_paese come tre valori di ingresso della funzione.

In [ ]:
grafico_integrato(*dati_paese('China'))

In [ ]:
grafico_integrato(*dati_regione('Puglia'))

## Dati incrementali

Se la visualizzazione dei dati cumulativi dà un'idea importante dell'andamento dell'epidemia, un indicatore molto importante sono i dati incrementali: quanti più pazienti ci sono rispetto al giorno precedente. Quando questo indicatore raggiunge lo zero, vuol dire che l'epidemia è finita e non ci sono più nuovi casi. I dati sui nuovi confermati sono disponibili in alcune basi dati, ma comunque si possono ottenere facilmente tramite una semplice sottrazione. Purtroppo però prima è necessaria una conversione dei dati. Fino ad adesso abbiamo trattato serie di dati che contenevano l'indicazione della data al loro interno, e se sottraessimo questi dati, il sistema appaierebbe automaticamente le date e non ci farebbe ottenere il risultato desiderato.

In [ ]:
incremento_confermati_italia = np.array(confermati_italia[1:]) - np.array(confermati_italia[:-1])
plt.plot(incremento_confermati_italia)

Anche questa curva ha uno o più picchi, e oltrepassato questo picco i contagiati continuano ad aumentare, ma più lentamente. È normale che questa curva abbia un primo andamento crescente nella fase esponenziale, per poi stabilizzarsi nella fase lineare della crescita dei contagiati. Tuttavia questa curva non diventerà mai negativa perché i casi confermati non possono mai diminuire.
Un altro aspetto di questa curva è che presenta molti picchi locali ed è in generale piuttosto irregolare. Questo è dovuto al fatto che i casi vengono riportati in maniera irregolare e/o con ritardi. Per avere una curva più regolare, è utile eseguire un **filtraggio** di questi dati. Il modo più semplice di ottenerlo è prendere la differenza tra più giorni invece che tra un giorno e il precedente. Le oscillazioni presenti tra un giorno e l'altro verranno così soppresse. Questo ad esempio è il caso della curva con un periodo di 5 giorni.

In [ ]:
giorni_incremento = 5
incremento_confermati_italia = (np.array(confermati_italia[giorni_incremento:]) - np.array(confermati_italia[:-giorni_incremento]))/giorni_incremento
plt.plot(incremento_confermati_italia)

Ovviamente la curva così filtrata è meno reattiva ai cambiamenti improvvisi, ma è anche più facile da interpretare in quanto riflette più i cambiamenti reali e meno quelli dovuti a errori dei dati. Definiamo una funzione per semplificare il calcolo di questa curva:

In [ ]:
def calcola_incremento(dati):
    giorni_incremento = 5
    return (np.array(dati[giorni_incremento:]) - np.array(dati[:-giorni_incremento]))/giorni_incremento

Si può anche rappresentare la variazione giornaliera dei casi attivi. In questo caso, le guarigioni (e i decessi) influenzeranno l'andamento. Questa curva può raggiungere lo zero, che sarà il punto in cui gli attivi inizieranno effettivamente a diminuire.

In [ ]:
incremento_attivi_italia = calcola_incremento(attivi_italia)
incremento_attivi_cina = calcola_incremento(attivi_cina)

incremento_italia = calcola_incremento(confermati_italia)
incremento_cina = calcola_incremento(confermati_cina)

plt.plot(incremento_italia, label='Italia')
plt.plot(incremento_cina, label='Cina')
plt.legend()
plt.title('Incrementi giornalieri dei confermati')

plt.figure()
plt.plot(incremento_attivi_italia, label='Italia')
plt.plot(incremento_attivi_cina, label='Cina')
plt.legend()
plt.title('Incrementi giornalieri degli attivi')

## Rappresentazioni dei dati alternative

Una rappresentazione molto interessante è quella proposta su: https://aatishb.com/covidtrends/ .
Questa rappresentazione ha sulle ascisse il numero totale di contagi e sulle ordinate il numero di nuovi contagi giornalieri, il tutto rappresentato in scala logaritmica. Questa rappresentazione funziona perché nella fase esponenziale, sia il numero di contagi totali che i nuovi contagi giornalieri sono esponenziali, di conseguenza i punti in cui la crescita è esponenziale si troveranno su una retta. Nel momento in cui le misure di contenimento iniziano a funzionare, i punti iniziano a deviare da questa retta. Questo si può ottenere semplicemente con i dati a nostra disposizione.

In [ ]:
# Il numero di punti degli incrementi è inferiore al numero dei punti originali, perché gli estremi non possono venire considerati
plt.plot(confermati_italia[-len(incremento_italia):], incremento_italia, label='Italia')
plt.plot(confermati_cina[-len(incremento_cina):], incremento_cina, label='Cina')
plt.xscale('log')
plt.yscale('log')
plt.xlim(100,None)
plt.legend()

Come al solito, definiamo una funzione per la rappresentazione di cui sopra.

In [ ]:
def plot_trend(dati, etichetta):
    incremento_dati = calcola_incremento(dati)
    plt.plot(dati[-len(incremento_dati):], incremento_dati, label=etichetta)

Disegnamo questa curva per alcune regioni italiane.

In [ ]:
regione = 'Puglia'
plot_trend(dati_regione(regione)[0], regione)
regione = 'Toscana'
plot_trend(dati_regione(regione)[0], regione)
regione = 'Lombardia'
plot_trend(dati_regione(regione)[0], regione)
plt.xscale('log')
plt.yscale('log')
plt.legend()

# Cosa abbiamo imparato

* Visualizzazione dei dati in scala logaritmica
* Calcolo dei pazienti attivi e artimetica tra dati
* Visualizzazione tramite diagrammi a barre e diagrammi a barre cumulativi
* Estrazione dei dati incrementali
    * Estrazione semplice
    * Filtraggio dei dati
* Rappresentazione degli incrementi in funzione del numero di contagiati 